In [3]:
import pyspark
import socket
import time
from datetime import datetime

from urllib.parse import urlparse

from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext

import socket, time

In [4]:
# sc = SparkContext()
spark =  SparkSession.builder.appName('trash_bin').master('local').getOrCreate()
ssc = StreamingContext(spark, 10 )
sqlContext = SQLContext(spark)

In [3]:

# 1. Read input stream from socket (by default, sockets contain raw strings, which we must then parse in a structured format)

HOST = '192.168.0.151'
# HOST = '127.0.0.1'
PORT = 65433
NYSE = True

# def getMasterNodeIP():
#   from urllib.parse import urlparse
#   try:
#     url_parsed = urlparse(sc.uiWebUrl)
#     ip = url_parsed.netloc.split(':')[0]
#     return ip
#   except Exception as exp:
#     print(exp)
#     return None

lines = spark.readStream.format("socket").option("host", HOST).option("port", PORT).load()


NameError: name 'json_schema' is not defined

In [7]:
if NYSE:
    print('NYSE')
    structured_stream = lines.select(\
        split(lines.value, ",")[0].alias("exchange"),\
        split(lines.value, ",")[1].alias("company"),\
        split(lines.value, ",")[2].alias("date"),\
        split(lines.value, ",")[3].cast('Float').alias("price_open"),\
        split(lines.value, ",")[4].cast('Float').alias("price_high"),\
        split(lines.value, ",")[5].cast('Float').alias("price_low"),\
        split(lines.value, ",")[6].cast('Float').alias("price_close"),\
        split(lines.value, ",")[7].cast('Int').alias("stock_volume"),\
        split(lines.value, ",")[8].cast('Float').alias("price_adj_close"),\
        split(lines.value, ",")[9].cast('Timestamp').alias("timestamp"))

NYSE


In [8]:
structured_stream

DataFrame[exchange: string, company: string, date: string, price_open: float, price_high: float, price_low: float, price_close: float, stock_volume: int, price_adj_close: float, timestamp: timestamp]

In [5]:
if NYSE == False:
    print('trash_bin')
    structured_stream = lines.select(\
        split(lines.value, ",")[0].cast('string').alias("canton"),\
        split(lines.value, ",")[1].cast('string').alias("canton_code"),\
        split(lines.value, ",")[2].cast('string').alias("city"),\
        split(lines.value, ",")[3].cast('string').alias("country"),\
        split(lines.value, ",")[4].cast('string').alias("country_code"),\
        split(lines.value, ",")[5].cast('int').alias("filling_level"),\
        split(lines.value, ",")[6].cast('int').alias("id"),\
        split(lines.value, ",")[7].cast('double').alias("lat"),\
        split(lines.value, ",")[8].cast('double').alias("lon"),\
        split(lines.value, ",")[9].cast('boolean').alias("maintenance"),\
        split(lines.value, ",")[11].cast('string').alias("street"),\
        split(lines.value, ",")[12].cast('string').alias("timestamp"),\
        split(lines.value, ",")[13].cast('boolean').alias("update"),\
        split(lines.value, ",")[13].cast('boolean').alias("level_50")) 

In [9]:
windowed_stream = structured_stream.groupBy(window("timestamp", "4 seconds", "4 seconds"))

In [10]:
if NYSE:
    aggregation_stream = windowed_stream.agg(avg("price_high").alias("window_price_average"),
                                           sum("price_high").alias("price_sum"), count("timestamp").alias("count_prices"))
else: 
    aggregation_stream = windowed_stream.agg(avg("filling_level").alias("window_filling_level_average"))

In [11]:
# NOTE: always use 'complete' outputMode for aggregations and 'append' outputMode to get complete records (the entire stream)

streamingETLQuery = aggregation_stream.writeStream.format("memory").queryName("aggDF").outputMode("complete").start()

In [13]:
# %sql

spark.sql('SELECT window.start, window.end, window_price_average FROM aggDF')
# SELECT * from aggDF

query_result = spark.sql('SELECT * FROM aggDF')

In [14]:
query_result.head()

In [ ]:
query_result.head()

In [ ]:
import math
import sys
from pyspark.sql.functions import desc

#TODO: complete here the SQL statement required to get all the contents of the stream (use the name assigned earlier)
df = spark.sql("select * from aggDF")
print("Running SQL (initialization may take a while)...")

iter = 1

# initialize the historic average with some random value
prev_avg = 1.0
old_count = 0

while True:
  
  # only start computing once some data has been collected (note that the dataframe is automatically updated by Spark)
  if(df.count() != 0):
    while(old_count == df.count()):
      # don't do anything while there is no new data
      continue
      
    # update the total count in order to be able to use the condition above in the next iteration
    old_count = df.count()
    
    print("*********************************************************************************************************")
    print("\nIteration no. "+ str(iter) + "\n")
    print("Sample data from streaming dataframe:\n")
    
    # set the time to be the end of the window (just to have a visual indication of time)
    df = df.withColumn("time", df.window.end)
       
    # we sort by time first, because by default there is no ordering in the streaming dataframe and we want to only show the most recent results
    # the following allows showing the most recent 5 windows (the False parameter instructs Spark not to truncate the output)
    df = df.sort(desc("time"))
    df.show(5, False)

    print("Current number of windows processed in stream: "+ str(old_count) + "\n")

    #TODO: use here the field that denotes the sum of prices per window.
    # In order to get the overall historic sum, you must add all the prices per window (use a new aggregate to do this)
    val = df.select('price_sum').agg(sum('price_sum')).collect()[0][0]

    #TODO: the same for the counts of all items with prices
    counts = df.select('count_prices').agg(sum('count_prices')).collect()[0][0]
    if(counts != None and counts != 0):
      # update the historic average taking into account the new aggregated values
      historic_avg = val/counts
     
   
    # the following lines just handle pretty-printing the output...
    for indent in range(iter - 1):
      sys.stdout.write("|****|")
    print ("|----|  Average price before current window: " + str(prev_avg))
    sys.stdout.flush()
    
    #TODO: take the most recent window average - use the field name you assigned for the average price per window
    # since we already ordered by time, you can simply take the first element of the result
    current_avg = df.select("window_price_average").take(1)[0][0]
    
    # the following lines just handle pretty-printing the output...
    for indent in range(iter - 1):
      sys.stdout.write("      ") 
    sys.stdout.write("|****| " + " Current window average price: " + str(current_avg))
    print

    # compute the percentage change from the historic average up until this window (ignoring the first iteration, where the historic average is a random number)
    if((math.fabs(current_avg - prev_avg)/prev_avg) > 0.3 and iter > 1):
        print("\n\t!!! ANOMALY DETECTED: price fluctuated by "+ str(float("{0:.2f}".format(math.fabs(current_avg - prev_avg)/prev_avg)) * 100) + " % !!!\n\n\n")
        
    prev_avg = historic_avg
    iter += 1